In [1]:
from reader_utils.corpus_builder_utils import *
from note_utils.pitch_dictionary import PitchDictionary
from reader_utils.dataset_builder import DatasetBuilder
from model_utils.model_utils import *
from gensim.models import Word2Vec

In [2]:
pd = PitchDictionary("dataset_objects/pitches_dict.txt")
db = DatasetBuilder("dataset_objects/full_corpus")

In [3]:
vocab_size = pd.get_vocabulary_size()
window_size = 4 # 2, 4
num_ns = 10 # 2, 5, 10
embedding_layer_name = "n2v_embedding"

In [4]:
corpus = db.get_full_corpus(drop_limit=500)